In [70]:
import pandas as pd
import numpy as np
import glob
import gzip
import shutil
import os
from tqdm import tqdm
import plotly.express as px
import sklearn

# set infinite display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [71]:
# import the TPM dataframe
TPM = pd.read_csv('../../results/TPM.tsv', sep='\t', index_col=0)
# preview the dataframe only showing the first 10 columns
TPM.iloc[:, :10].head()

,52aef68b-63de-46b1-bb88-fb11d875c803,98460c61-c4ef-43ff-b7ac-b5c1b7b0ae26,5810cc10-f53a-451a-8001-c68765e30565,2042bb7e-ff24-439d-b33c-70c15cc3d201,dba512d3-58c5-47ed-a3f5-ee3d81e0b0c9,4aa493c6-6177-4d64-add9-f0c0198e2530,53b68f98-1d15-42d5-b374-b2163590b3a8,1748692d-651e-4bb1-ab9e-8460d1c8b588,eb0750ce-bed8-41a0-b93c-d72d02c65303,cf4aa3e0-2256-42e3-a7cf-9ea08900b622
Name,,,,,,,,,,
ENST00000456328.2,0.088142,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.147231
ENST00000450305.2,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENST00000488147.1,3.235923,2.721829,5.28927,13.728161,2.106743,6.399453,2.024906,1.171819,0.643958,4.371894
ENST00000619216.1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENST00000473358.1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [72]:
# print shape of dataframe
print('Before removing non-performing transcripts: ', TPM.shape)
# find genes with 0 TPM in all samples
TPM = TPM.loc[(TPM != 0).any(axis=1)]
# print shape of dataframe
print('After removing non-performing transcripts: ', TPM.shape)

# # find genes with 0 TPM in at least 99% of samples
# TPM = TPM.loc[(TPM == 0).sum(axis=1) <= 0.99 * TPM.shape[1]]
# # print shape of dataframe
# print('After removing genes with 0 TPM in at least 99% of samples: ', TPM.shape)

Before removing non-performing transcripts:  (252045, 133)
After removing non-performing transcripts:  (233502, 133)


In [73]:
# transpose the dataframe
TPM = TPM.T

In [74]:
# Flatten the DataFrame to a 1D array
flat_values = TPM.values.flatten()

# Calculate mean and standard deviation
mean_value = np.mean(flat_values)
std_value = np.std(flat_values)

# calculate max, min and range
max_value = np.max(flat_values)
min_value = np.min(flat_values)
range_value = max_value - min_value

# print all statistical values
print('Mean TPM: ', mean_value)
print('Standard deviation TPM: ', std_value)
print('Max TPM: ', max_value)
print('Min TPM: ', min_value)
print('Range TPM: ', range_value)

# find the smallest non-zero value in the flattened TPM array
smallest_nonzero = np.min(TPM.values[TPM.values > 0])
# print the smallest non-zero value
print('Smallest non-zero TPM value: ', smallest_nonzero)

Mean TPM:  4.282618564281844
Standard deviation TPM:  657.9168415005502
Max TPM:  366584.136888
Min TPM:  0.0
Range TPM:  366584.136888
Smallest non-zero TPM value:  1e-06


In [75]:
# introduce pseudocount
TPM = TPM + 0.000000001
# perform log-transformation
TPM = TPM.apply(np.log2)
# preview the dataframe only showing the first 10 columns
TPM.iloc[:, :10].head()

Name,ENST00000456328.2,ENST00000488147.1,ENST00000473358.1,ENST00000469289.1,ENST00000417324.1,ENST00000461467.1,ENST00000606857.1,ENST00000642116.1,ENST00000466430.5,ENST00000477740.5
52aef68b-63de-46b1-bb88-fb11d875c803,-3.504027,1.694177,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-2.315795,-29.897353
98460c61-c4ef-43ff-b7ac-b5c1b7b0ae26,-29.897353,1.444576,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-3.405161,-29.897353
5810cc10-f53a-451a-8001-c68765e30565,-29.897353,2.403069,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-2.343447,-29.897353
2042bb7e-ff24-439d-b33c-70c15cc3d201,-29.897353,3.779066,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-1.387547,-29.897353
dba512d3-58c5-47ed-a3f5-ee3d81e0b0c9,-29.897353,1.075014,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-29.897353,-3.771008,-29.897353


In [76]:
# perform PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(TPM)

# transform the data
TPM_pca = pca.transform(TPM)

# create a dataframe with the PCA results
TPM_pca_df = pd.DataFrame(data = TPM_pca, columns = ['PC1', 'PC2', 'PC3'], index=TPM.index)

In [77]:
# preview the dataframe
TPM_pca_df.head()

,PC1,PC2,PC3
52aef68b-63de-46b1-bb88-fb11d875c803,-990.899027,253.856767,779.595577
98460c61-c4ef-43ff-b7ac-b5c1b7b0ae26,-350.295736,566.340550,-1081.627274
5810cc10-f53a-451a-8001-c68765e30565,4934.359389,-297.486593,-735.512843
2042bb7e-ff24-439d-b33c-70c15cc3d201,4734.320720,-933.215457,-221.813267
dba512d3-58c5-47ed-a3f5-ee3d81e0b0c9,-304.568836,-456.593663,659.383070


In [78]:
# use plotly to create a 2d scatterplot
fig = px.scatter(TPM_pca_df, x='PC1', y='PC2', title='PCA of TPM values')
fig.show()

In [79]:
# use plotly to create a 3d scatterplot
fig = px.scatter_3d(TPM_pca_df, x='PC1', y='PC2', z='PC3', title='PCA of TPM values')
fig.show()

In [80]:
# import the metadata
metadata = pd.read_csv('../../results/metadata.tsv', sep='\t', index_col=0)
# preview the dataframe
metadata.head()

,aliquot_id,read_group_id,has_blood_cancer,tissue_type,instrument_model,RIN,includes_spike_ins,library_preparation_kit_name,library_preparation_kit_vendor,library_preparation_kit_version,library_preparation_kit_catalog_number,library_selection,library_strand,library_strategy,size_selection_range,spike_ins_concentration,demographic_id,diagnosis_id,batch_id,index_date,cause_of_death,days_to_birth,days_to_death,ethnicity,gender,race,vital_status,adrenal_hormone,age_at_diagnosis,ajcc_clinical_m,ajcc_clinical_n,ajcc_clinical_stage,ajcc_clinical_t,ajcc_pathologic_m,ajcc_pathologic_n,ajcc_pathologic_stage,ajcc_pathologic_t,ajcc_staging_system_edition,ann_arbor_b_symptoms,ann_arbor_b_symptoms_described,ann_arbor_clinical_stage,ann_arbor_extranodal_involvement,ann_arbor_pathologic_stage,best_overall_response,burkitt_lymphoma_clinical_variant,child_pugh_classification,classification_of_tumor,cog_liver_stage,cog_neuroblastoma_risk_group,cog_renal_stage,cog_rhabdomyosarcoma_risk_group,created_datetime,days_to_best_overall_response,days_to_diagnosis,days_to_last_follow_up,days_to_last_known_disease_status,days_to_recurrence,diagnosis_is_primary_disease,eln_risk_classification,enneking_msts_grade,enneking_msts_metastasis,enneking_msts_stage,enneking_msts_tumor_site,esophageal_columnar_dysplasia_degree,esophageal_columnar_metaplasia_present,figo_stage,figo_staging_edition_year,first_symptom_prior_to_diagnosis,gastric_esophageal_junction_involvement,gleason_grade_group,gleason_grade_tertiary,gleason_patterns_percent,goblet_cells_columnar_mucosa_present,icd_10_code,igcccg_stage,inpc_grade,inpc_histologic_group,inrg_stage,inss_stage,international_prognostic_index,irs_group,irs_stage,ishak_fibrosis_score,iss_stage,last_known_disease_status,laterality,margin_distance,margins_involved_site,masaoka_stage,medulloblastoma_molecular_classification,metastasis_at_diagnosis,metastasis_at_diagnosis_site,method_of_diagnosis,micropapillary_features,mitosis_karyorrhexis_index,mitotic_count,morphology,ovarian_specimen_status,ovarian_surface_involvement,papillary_renal_cell_type,peritoneal_fluid_cytological_status,pregnant_at_diagnosis,primary_diagnosis,primary_disease,primary_gleason_grade,prior_malignancy,prior_treatment,progression_or_recurrence,project_id.1,residual_disease,satellite_nodule_present,secondary_gleason_grade,site_of_resection_or_biopsy,sites_of_involvement,submitter_id,supratentorial_localization,synchronous_malignancy,tissue_or_organ_of_origin,tumor_confined_to_organ_of_origin,tumor_depth,tumor_focality,tumor_grade,tumor_regression_grade,updated_datetime,weiss_assessment_score,who_cns_grade,who_nte_grade,wilms_tumor_histologic_subtype,year_of_diagnosis,disease_type,primary_site,project_id
case_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5705efcc-b48f-435c-8a28-9e0d407ecadd,75ac0619-947a-427b-a53f-71e121a7ec8f,71894d8b-5210-44dc-aadc-a199d3843dd2,False,Tumor,Illumina HiSeq 4000,NaN,True,TruSeq Stranded Total RNA Library Prep Kit wit...,Illumina,NaN,NaN,rRNA Depletion,First_Stranded,RNA-Seq,308.0,1.0,f1ec7b5b-7f11-41e6-9fb4-a27fb2adc8f1,c7f325cd-98a6-4996-825e-1c23b86e1a22,2.0,Diagnosis,NaN,-22142.0,NaN,not hispanic or latino,female,white,Alive,NaN,22142.0,M0,NaN,NaN,NaN,Unknown,NX,Stage I,T1a,7th,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-17T23:34:39.302537-05:00,NaN,NaN,823.0,823.0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stage I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tumor free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8380/3,NaN,NaN,NaN,NaN,NaN,"Endometrioid adenocarcinoma, NOS",NaN,NaN,NaN,NaN,no,CPTAC-3,R0,NaN,NaN,Corpus uteri,NaN,C3L-00137-DIAG,NaN,NaN,Corpus uteri,NaN,NaN,Unifocal,G1,NaN,2021-09-22T14:36:32.037275-05:00,NaN,NaN,NaN,NaN,2016.0,Adenomas and Adenocarcinomas,"Uterus, NOS",CPTAC-3
5705efcc-b48f-435c-8a28-9e0d407ecadd,948c4d53-3d91-48a6-bec4-0cc96020e572,86774648-bb57-42c3-b835-9fb11b590d

In [81]:
# give the index column the name 'read_group_id'
TPM_pca_df.index.name = 'read_group_id'
# transform the index header of TPM_pca_df into a column
TPM_pca_df.reset_index(inplace=True)

In [83]:
# preview the dataframe
TPM_pca_df.head()

,read_group_id,PC1,PC2,PC3
0,52aef68b-63de-46b1-bb88-fb11d875c803,-990.899027,253.856767,779.595577
1,98460c61-c4ef-43ff-b7ac-b5c1b7b0ae26,-350.295736,566.340550,-1081.627274
2,5810cc10-f53a-451a-8001-c68765e30565,4934.359389,-297.486593,-735.512843
3,2042bb7e-ff24-439d-b33c-70c15cc3d201,4734.320720,-933.215457,-221.813267
4,dba512d3-58c5-47ed-a3f5-ee3d81e0b0c9,-304.568836,-456.593663,659.383070


In [84]:
# take tissue_type, gender, race, age_at_diagnosis/365, ajcc_pathologic_stage, primary_diagnosis, morphology, tissue_or_organ_of_origin, tumor_focality, disease_type, primary_site from the metadata and merge it with the TPM_pca_df into a new dataframe called TPM_pre_cluster
columns = ['read_group_id', 'tissue_type', 'gender', 'race', 'age_at_diagnosis', 'ajcc_pathologic_stage', 'primary_diagnosis', 'morphology', 'tissue_or_organ_of_origin', 'tumor_focality', 'disease_type', 'primary_site']

# Create a new dataframe with selected columns from metadata
selected_metadata = metadata[columns]

# Merge selected_metadata with TPM_pca_df using the 'read_group_id' column
TPM_pre_cluster = pd.merge(TPM_pca_df, selected_metadata, on='read_group_id')

In [85]:
# divide age_at_diagnosis by 365 to get age_at_diagnosis in years
TPM_pre_cluster['age_at_diagnosis'] = TPM_pre_cluster['age_at_diagnosis'] / 365

In [86]:
# preview the dataframe
TPM_pre_cluster.head()

,read_group_id,PC1,PC2,PC3,tissue_type,gender,race,age_at_diagnosis,ajcc_pathologic_stage,primary_diagnosis,morphology,tissue_or_organ_of_origin,tumor_focality,disease_type,primary_site
0,52aef68b-63de-46b1-bb88-fb11d875c803,-990.899027,253.856767,779.595577,Tumor,female,asian,39.980822,Not Reported,Glioblastoma,9440/3,"Brain, NOS",NaN,Gliomas,Brain
1,98460c61-c4ef-43ff-b7ac-b5c1b7b0ae26,-350.295736,566.340550,-1081.627274,Tumor,male,white,65.978082,Stage IVA,"Squamous cell carcinoma, NOS",8070/3,"Tongue, NOS",Unifocal,Squamous Cell Neoplasms,Other and ill-defined sites
2,5810cc10-f53a-451a-8001-c68765e30565,4934.359389,-297.486593,-735.512843,Tumor,male,white,72.452055,Unknown,"Squamous cell carcinoma, NOS",8070/3,"Larynx, NOS",Unifocal,Squamous Cell Neoplasms,Other and ill-defined sites
3,2042bb7e-ff24-439d-b33c-70c15cc3d201,4734.320720,-933.215457,-221.813267,Tumor,female,white,42.550685,Stage I,"Endometrioid adenocarcinoma, NOS",8380/3,Endometrium,Unifocal,Adenomas and Adenocarcinomas,"Uterus, NOS"
4,dba512d3-58c5-47ed-a3f5-ee3d81e0b0c9,-304.568836,-456.593663,659.383070,Normal,female,white,39.065753,Stage I,"Endometrioid adenocarcinoma, NOS",8380/3,Endometrium,Unifocal,Adenomas and Adenocarcinomas,"Uterus, NOS"


In [87]:
# remove read_group_id column
TPM_pre_cluster.drop('read_group_id', axis=1, inplace=True)

In [89]:
# do k-means clustering on the pre-clustered dataframe
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=0).fit(TPM_pre_cluster.iloc[:, :3])

# add the cluster labels to the pre-clustered dataframe
TPM_pre_cluster['cluster'] = kmeans.labels_

# use plotly to create a 3d scatterplot
fig = px.scatter_3d(TPM_pre_cluster, x='PC1', y='PC2', z='PC3', color='cluster', title='PCA of TPM values')

/Users/worker/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

